<a href="https://colab.research.google.com/github/cpython-projects/da_1709/blob/main/lesson_25_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `CASE WHEN`

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import plotly.express as px

In [2]:
DB_USER = "prog_academy_da_yuq6_user"
DB_PASS = "nKLaf3orGpgat1GEpFbkuNVtxcNAGJp1"
DB_HOST = "dpg-d519e97pm1nc73briqk0-a.oregon-postgres.render.com"
DB_PORT = "5432"
DB_NAME = "prog_academy_da_yuq6"

In [3]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

**`CASE WHEN`** — це умовний оператор у SQL, аналог `if / else`, який дозволяє
повертати різні значення залежно від виконання умов.

Він **не є віконною функцією**, але **дуже часто використовується разом із ними** для:

* створення ознак (features)
* сегментації користувачів
* класифікації подій

---

## Базовий синтаксис

```sql
CASE
    WHEN умова_1 THEN результат_1
    WHEN умова_2 THEN результат_2
    ELSE результат_за_замовчуванням
END
```

📌 Перевірка йде **зверху вниз** — виконується **перша істинна умова**.

---

## Простий приклад

```sql
SELECT total_uah,
       CASE
           WHEN total_uah >= 200 THEN 'high'
           WHEN total_uah >= 100 THEN 'medium'
           ELSE 'low'
       END AS order_size
FROM orders_log;
```

**Логіка:**

* якщо сума ≥ 200 → `high`
* інакше якщо ≥ 100 → `medium`
* інакше → `low`

---

## `CASE WHEN` + віконні функції

Найпоширеніший сценарій — **класифікація на основі результату window-функції**.

### Приклад: перша чи повторна покупка

```sql
SELECT user_uuid, order_time, total_uah,
       CASE
           WHEN ROW_NUMBER() OVER (
                    PARTITION BY user_uuid
                    ORDER BY order_time
                ) > 1 THEN 'R'
           ELSE 'F'
       END AS purchase_type
FROM orders_log;
```

### Як це працює по кроках:

1. `ROW_NUMBER()` нумерує замовлення користувача
2. `CASE WHEN` перевіряє номер:

   * `1` → перша покупка (`F`)
   * `>1` → повторна (`R`)
3. Результат — **новий аналітичний стовпець**

## Обов’язковий `ELSE`

Якщо **не вказати `ELSE`**, SQL поверне `NULL`.

```sql
CASE
    WHEN total_uah > 100 THEN 'big'
END
```

📌 Якщо жодна умова не виконалась → `NULL`

**Рекомендація:** завжди явно вказувати `ELSE`.

---

## Типи даних у `CASE`

Усі `THEN` і `ELSE` мають повертати **сумісні типи**:

❌ Неправильно:

```sql
CASE
    WHEN total_uah > 100 THEN 'big'
    ELSE 0
END
```

✅ Правильно:

```sql
CASE
    WHEN total_uah > 100 THEN 1
    ELSE 0
END
```

або

```sql
CASE
    WHEN total_uah > 100 THEN 'big'
    ELSE 'small'
END
```

---

## Коли використовувати `CASE WHEN`

*  створення сегментів  
*  логічні прапорці (0/1, Y/N, F/R)  
*  категоризація чисел  
*  обробка результатів window-функцій  

**Завдання 1. Ознака повторної покупки**  
Чи зробив користувач щонайменше 2 замовлення?

**Завдання 2. Тип користувача: one-time / repeat / heavy**  
До якого типу належить користувач за кількістю замовлень?

**Завдання 3. Перше замовлення чи повторне**  
Чи є це замовлення першим для користувача?

**Завдання 4. Зростання або падіння чека**  
Як змінився чек порівняно з попередньою покупкою?

**Завдання 5. Замовлення вище середнього для користувача**  
Чи є це замовлення вищим за середній чек користувача?

**Завдання 6. Ознака відтоку користувача (churn flag)**  
Користувач не робив покупок більше N днів (наприклад, 30)?